In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast

import warnings

warnings.filterwarnings('ignore')

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'backing_material'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Rugs & Mats"""


# regex_pattern = r"(?i)(\s[^t]\d+\-Inch x \d+\-Inch)|()"
regex_pattern = r'(\w*\s?backing)|()'


value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [8]:
df = query_from_file(file_name='query/curated_products_VBOutDiamMM.sql', params=params)
df

attribute  value     customer_name external_id  \
0      backing_material  Vinyl  bedbathandbeyond    14397132   
1      backing_material  Vinyl  bedbathandbeyond    17617605   
2      backing_material  Vinyl  bedbathandbeyond    17619667   
3      backing_material  Vinyl  bedbathandbeyond    17664433   
4      backing_material  Vinyl  bedbathandbeyond    18076136   
...                 ...    ...               ...         ...   
98343  backing_material   Felt  bedbathandbeyond    68117863   
98344  backing_material   Felt  bedbathandbeyond    68117887   
98345  backing_material   Felt  bedbathandbeyond    68117917   
98346  backing_material   Felt  bedbathandbeyond    68118075   
98347  backing_material   Felt  bedbathandbeyond    68309565   

                                                     long_desc resolution  
0                                 Rocks Vinyl Tub Mat in Clear   standard  
1                  University of Texas 19\" x 30\" Starter Mat   standard  
2                Texas Tech University 19\" x 30\" Starter Mat   standard  
3                 Texas A&M University 19\" x 30\" Starter Mat   standard  
4             Bungalow Flooring DirtStopper Small Rug in Brown   standard  
...                                                        ...        ...  
98343     Couristan® Chalet Oasis 3'4 x 5'3 Area Rug in Pastel   standard  
98344      Couristan® Chalet Oasis 5'4 x 8' Area Rug in Pastel   standard  
98345     Couristan® Chalet Oasis 8' x 11'4 Area Rug in Pastel   standard  
98346  Couristan® Chalet Celestial 8' x 11'4 Area Rug in Brown   standard  
98347           Jaipur Living Paxton Gray/Ivory Rug (5'X7'6\")   standard  

[98348 rows x 6 columns]

In [9]:
metric='in'

range_string= """
 101 in - 105 in
106 in - 110 in
111 in - 115 in
116 in - 120 in
121 in - 125 in
126 in - 130 in
131 in - 135 in
136 in - 140 in
20 in - 25 in
26 in - 30 in
31 in - 35 in
36 in - 40 in
41 in - 45 in
46 in - 50 in
51 in - 55 in
56 in - 60 in
61 in - 65 in
66 in - 70 in
71 in - 75 in
76 in - 80 in
81 in - 85 in
86 in - 90 in
91 in - 95 in
96 in - 100 in
Above 140 in
Under 20 in 
"""

In [10]:
# # Find a list of values to iterate
# df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
# df_strategy
# list_values=df_strategy['value'].to_list()
# list_values

In [11]:
# kwp=KeywordProcessor()
# print(kwp.add_keywords_from_list(list_values))
# # Extract Key Words
# df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

In [12]:
# df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
# df['matches'].explode().value_counts()

In [27]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df

attribute  value     customer_name external_id  \
0      backing_material  Vinyl  bedbathandbeyond    14397132   
1      backing_material  Vinyl  bedbathandbeyond    17617605   
2      backing_material  Vinyl  bedbathandbeyond    17619667   
3      backing_material  Vinyl  bedbathandbeyond    17664433   
4      backing_material  Vinyl  bedbathandbeyond    18076136   
...                 ...    ...               ...         ...   
98343  backing_material   Felt  bedbathandbeyond    68117863   
98344  backing_material   Felt  bedbathandbeyond    68117887   
98345  backing_material   Felt  bedbathandbeyond    68117917   
98346  backing_material   Felt  bedbathandbeyond    68118075   
98347  backing_material   Felt  bedbathandbeyond    68309565   

                                                     long_desc resolution  \
0                                 Rocks Vinyl Tub Mat in Clear   standard   
1                  University of Texas 19\" x 30\" Starter Mat   standard   
2                Texas Tech University 19\" x 30\" Starter Mat   standard   
3                 Texas A&M University 19\" x 30\" Starter Mat   standard   
4             Bungalow Flooring DirtStopper Small Rug in Brown   standard   
...                                                        ...        ...   
98343     Couristan® Chalet Oasis 3'4 x 5'3 Area Rug in Pastel   standard   
98344      Couristan® Chalet Oasis 5'4 x 8' Area Rug in Pastel   standard   
98345     Couristan® Chalet Oasis 8' x 11'4 Area Rug in Pastel   standard   
98346  Couristan® Chalet Celestial 8' x 11'4 Area Rug in Brown   standard   
98347           Jaipur Living Paxton Gray/Ivory Rug (5'X7'6\")   standard   

      matches  
0          []  
1          []  
2          []  
3          []  
4          []  
...       ...  
98343      []  
98344      []  
98345      []  
98346      []  
98347      []  

[98348 rows x 7 columns]

In [28]:
df[200:250]

attribute  value     customer_name external_id  \
200  backing_material  Vinyl  bedbathandbeyond    66517795   
201  backing_material  Vinyl  bedbathandbeyond    66517825   
202  backing_material  Vinyl  bedbathandbeyond    66517832   
203  backing_material  Vinyl  bedbathandbeyond    66517856   
204  backing_material  Vinyl  bedbathandbeyond    66517870   
205  backing_material  Vinyl  bedbathandbeyond    66517900   
206  backing_material  Vinyl  bedbathandbeyond    66517962   
207  backing_material  Vinyl  bedbathandbeyond    66518020   
208  backing_material  Vinyl  bedbathandbeyond    66518082   
209  backing_material  Vinyl  bedbathandbeyond    66518174   
210  backing_material  Vinyl  bedbathandbeyond    66518204   
211  backing_material  Vinyl  bedbathandbeyond    66518228   
212  backing_material  Vinyl  bedbathandbeyond    66529316   
213  backing_material  Vinyl  bedbathandbeyond    66529354   
214  backing_material  Vinyl  bedbathandbeyond    66529392   
215  backing_material  Vinyl  bedbathandbeyond    66529408   
216  backing_material  Vinyl  bedbathandbeyond    66529415   
217  backing_material  Vinyl  bedbathandbeyond    66529439   
218  backing_material  Vinyl  bedbathandbeyond    66529446   
219  backing_material  Vinyl  bedbathandbeyond    67221370   
220  backing_material  Vinyl  bedbathandbeyond    67221431   
221  backing_material  Vinyl  bedbathandbeyond    67221455   
222  backing_material  Vinyl  bedbathandbeyond    67221479   
223  backing_material  Vinyl  bedbathandbeyond    67221486   
224  backing_material  Vinyl  bedbathandbeyond    67221509   
225  backing_material  Vinyl  bedbathandbeyond    67221530   
226  backing_material  Vinyl  bedbathandbeyond    67221547   
227  backing_material  Vinyl  bedbathandbeyond    67221554   
228  backing_material  Vinyl  bedbathandbeyond    67221561   
229  backing_material  Vinyl  bedbathandbeyond    67221592   
230  backing_material  Vinyl  bedbathandbeyond    67221615   
231  backing_material  Vinyl  bedbathandbeyond    67221622   
232  backing_material  Vinyl  bedbathandbeyond    67221660   
233  backing_material  Vinyl  bedbathandbeyond    67221677   
234  backing_material  Vinyl  bedbathandbeyond    67221691   
235  backing_material  Vinyl  bedbathandbeyond    67221714   
236  backing_material  Vinyl  bedbathandbeyond    67221721   
237  backing_material  Vinyl  bedbathandbeyond    67221738   
238  backing_material  Vinyl  bedbathandbeyond    67221745   
239  backing_material  Vinyl  bedbathandbeyond    67221769   
240  backing_material  Vinyl  bedbathandbeyond    67221820   
241  backing_material  Vinyl  bedbathandbeyond    67221837   
242  backing_material  Vinyl  bedbathandbeyond    68684785   
243  backing_material  Vinyl  bedbathandbeyond    69296789   
244  backing_material  Vinyl  bedbathandbeyond    69359941   
245  backing_material  Vinyl  bedbathandbeyond    75983769   
246  backing_material  Vinyl  bedbathandbeyond    75983784   
247  backing_material  Vinyl  bedbathandbeyond    75983795   
248  backing_material  Vinyl  bedbathandbeyond    76246291   
249  backing_material  Vinyl  bedbathandbeyond    76477723   

                                                                       long_desc  \
200           Calloway Mills Christmas Trio 17\" x 29\" Multicolor Coir Door Mat   
201          Calloway Mills Holiday Snowman 17\" x 29\" Multicolor Coir Door Mat   
202               Calloway Mills Jingle 17\" x 29\" Coir Door Mat in Red/Natural   
203               Calloway Mills Bon Voyage 17\" x 29\" Multicolor Coir Door Mat   
204               Calloway Mills Pirate 17\" x 29\" Coir Door Mat in Natural/Red   
205                 Calloway Mills Bicycles 17\" x 29\" Multicolor Coir Door Mat   
206         Calloway Mills Home of the Free 17\" x 29\" Multicolor Coir Door Mat   
207            Calloway Mills Triple Stripe 17\" x 29\" Multicolor Coir Door Mat   
208         Calloway Mills Colorful Daisies 17\" x 29\" Multicolor Coir Door Mat   
209  Call

In [29]:
df[df['value'].astype(str)!='[]']

attribute  value     customer_name external_id  \
0      backing_material  Vinyl  bedbathandbeyond    14397132   
1      backing_material  Vinyl  bedbathandbeyond    17617605   
2      backing_material  Vinyl  bedbathandbeyond    17619667   
3      backing_material  Vinyl  bedbathandbeyond    17664433   
4      backing_material  Vinyl  bedbathandbeyond    18076136   
...                 ...    ...               ...         ...   
98343  backing_material   Felt  bedbathandbeyond    68117863   
98344  backing_material   Felt  bedbathandbeyond    68117887   
98345  backing_material   Felt  bedbathandbeyond    68117917   
98346  backing_material   Felt  bedbathandbeyond    68118075   
98347  backing_material   Felt  bedbathandbeyond    68309565   

                                                     long_desc resolution  \
0                                 Rocks Vinyl Tub Mat in Clear   standard   
1                  University of Texas 19\" x 30\" Starter Mat   standard   
2                Texas Tech University 19\" x 30\" Starter Mat   standard   
3                 Texas A&M University 19\" x 30\" Starter Mat   standard   
4             Bungalow Flooring DirtStopper Small Rug in Brown   standard   
...                                                        ...        ...   
98343     Couristan® Chalet Oasis 3'4 x 5'3 Area Rug in Pastel   standard   
98344      Couristan® Chalet Oasis 5'4 x 8' Area Rug in Pastel   standard   
98345     Couristan® Chalet Oasis 8' x 11'4 Area Rug in Pastel   standard   
98346  Couristan® Chalet Celestial 8' x 11'4 Area Rug in Brown   standard   
98347           Jaipur Living Paxton Gray/Ivory Rug (5'X7'6\")   standard   

      matches  
0          []  
1          []  
2          []  
3          []  
4          []  
...       ...  
98343      []  
98344      []  
98345      []  
98346      []  
98347      []  

[98348 rows x 7 columns]

In [30]:
# df=df[(df['value'].astype(str)=='n/a')&(df['matches'].astype(str)!='[]')]
# df

In [31]:
df['matches'].explode().value_counts()

Series([], Name: matches, dtype: int64)

In [18]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
    # data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())
    data[['length', 'width']]=data['matches'].str.split("x",expand=True,)

In [19]:
def drop_characters(data):
    data['length']=data['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['width']=data['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('"', ''))
    data['width']=data['width'].apply(lambda quote: quote.replace('"', ''))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('-',''))
    data['width']=data['width'].apply(lambda quote: quote.replace('-',''))
    data['width']=data['width'].apply(lambda x: 0 if x == '' else x)
    data['length']=data['length'].apply(lambda x: 0 if x == '' else x)

In [20]:
def add_quotes(data):
    data['length']=data['length'].apply(lambda x: str(x) + "'")
    data['width']=data['width'].apply(lambda x: str(x) + "'")


In [21]:
def min_max(data):
    data['length']=data['length'].astype(int)
    data['width']=data['width'].astype(int)
    data['Max'] = data[['length', 'width']].values.max(1)
    data['Min'] = data[['length', 'width']].values.min(1)

In [24]:
df

attribute  value     customer_name external_id  \
0      backing_material  Vinyl  bedbathandbeyond    14397132   
1      backing_material  Vinyl  bedbathandbeyond    17617605   
2      backing_material  Vinyl  bedbathandbeyond    17619667   
3      backing_material  Vinyl  bedbathandbeyond    17664433   
4      backing_material  Vinyl  bedbathandbeyond    18076136   
...                 ...    ...               ...         ...   
98343  backing_material   Felt  bedbathandbeyond    68117863   
98344  backing_material   Felt  bedbathandbeyond    68117887   
98345  backing_material   Felt  bedbathandbeyond    68117917   
98346  backing_material   Felt  bedbathandbeyond    68118075   
98347  backing_material   Felt  bedbathandbeyond    68309565   

                                                     long_desc resolution  \
0                                 Rocks Vinyl Tub Mat in Clear   standard   
1                  University of Texas 19\" x 30\" Starter Mat   standard   
2                Texas Tech University 19\" x 30\" Starter Mat   standard   
3                 Texas A&M University 19\" x 30\" Starter Mat   standard   
4             Bungalow Flooring DirtStopper Small Rug in Brown   standard   
...                                                        ...        ...   
98343     Couristan® Chalet Oasis 3'4 x 5'3 Area Rug in Pastel   standard   
98344      Couristan® Chalet Oasis 5'4 x 8' Area Rug in Pastel   standard   
98345     Couristan® Chalet Oasis 8' x 11'4 Area Rug in Pastel   standard   
98346  Couristan® Chalet Celestial 8' x 11'4 Area Rug in Brown   standard   
98347           Jaipur Living Paxton Gray/Ivory Rug (5'X7'6\")   standard   

       matches  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4          NaN  
...        ...  
98343      NaN  
98344      NaN  
98345      NaN  
98346      NaN  
98347      NaN  

[98348 rows x 7 columns]

In [23]:
split_col(df) #might throw an error and have to drop nan

AttributeError: Can only use .str accessor with string values!

In [ ]:
drop_characters(df)
min_max(df) #error base 10 when there is only a value for max and not min

In [ ]:
df

In [ ]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [ ]:
na_to_gsheet(df)

In [ ]:
# range function builder

string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)

In [ ]:
matches=df
matches

In [ ]:
matches[curation_col] = matches['Max'].apply(range_app)
matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

In [ ]:
## KWP
# matches[curation_col]=matches['matches']
# matches

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [ ]:
looks_good('Bed Bath & Beyond', attribute, df, matches)